In [1]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        680M        413M        992K         11G         11G
Swap:            0B          0B          0B


In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # !cd "/content/drive/MyDrive/Colab Notebooks"
# !unzip -u "/content/drive/MyDrive/Colab Notebooks/Benign.zip" -d "/content/drive/MyDrive/Colab Notebooks/dataset"
# !unzip -u "/content/drive/MyDrive/Colab Notebooks/Invasive.zip" -d "/content/drive/MyDrive/Colab Notebooks/dataset"
# !unzip -u "/content/drive/MyDrive/Colab Notebooks/Normal.zip" -d "/content/drive/MyDrive/Colab Notebooks/dataset"

In [3]:
import numpy as np # Xử lý một số toán học
import matplotlib.pyplot as plt # Thư viện đồ thị trực quan, hình ảnh,...
import pathlib
import os, sys
import cv2 
import random
import pickle
import tensorflow as tf
from tensorflow.data import AUTOTUNE
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, AveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
data_dir = "/content/drive/MyDrive/Colab Notebooks/dataset"
data_dir = pathlib.Path(data_dir)
categories = os.listdir(data_dir) 
number_classes = len(categories)

In [5]:
batch_size = 32
img_height = 1536
img_width = 2048
scale_percent = 25
width = int(img_width * scale_percent / 100)
height = int(img_height * scale_percent / 100)
dim = (width, height)

In [6]:
X = []
y = []

In [7]:
new_img_array = None

def create_training_data():
  for category in categories: 
    path = os.path.join(data_dir, category)  
    label_number = categories.index(category) 
    for img in os.listdir(path): 
    # try:
      img_array     = cv2.imread(os.path.join(path, img), cv2.IMREAD_ANYCOLOR) 
      new_img_array = cv2.resize(img_array, dim, interpolation = cv2.INTER_AREA) 
      new_img_array = np.array(new_img_array)

      X.append(new_img_array)  
      y.append(label_number)  
    # except Exception as e:
    #   pass
    print(category + " success!")

create_training_data()

Benign success!
Invasive success!
Normal success!


In [10]:
# Chạy 1 lần, thành công thì không chạy nữa
pickle_out = open("/content/drive/MyDrive/Colab Notebooks/X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

In [11]:
# Chạy 1 lần, thành công thì không chạy nữa
pickle_out = open("/content/drive/MyDrive/Colab Notebooks/y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()
# Chạy đc tới đây là coi như ổn

In [12]:
# Mỗi lần chạy mô hình thì chạy để load dữ liệu
pickle_in = open("/content/drive/MyDrive/Colab Notebooks/X.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open("/content/drive/MyDrive/Colab Notebooks/y.pickle", "rb")
y = pickle.load(pickle_in)
pickle_in.close()

In [13]:
X = np.array(X)

In [14]:
X.shape

(1119, 384, 512, 3)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X = y = []
X_train, X_val, y_train, y_val =  train_test_split(X_train, y_train, test_size=0.2, random_state=42)

y_train = to_categorical(y_train) # one hot
# ví dụ y_number = 14 thì y = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
y_t = y_test
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [16]:
 #  Khởi tạo một một mô hình neural
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(filters=32, kernel_size=(3,3), input_shape=X_train.shape[1:], padding="valid", activation="relu"),
  layers.MaxPooling2D(pool_size=(2, 2), strides=2),
  layers.Conv2D(64, 3, activation='tanh'),
  layers.MaxPooling2D(pool_size=(2, 2), strides=2),
  layers.Conv2D(128, 3, activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2), strides=2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(number_classes),
  layers.Activation("softmax")

])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=20, epochs=25)

model.summary()  

Epoch 1/25
36/36 [==============================] - 256s 7s/step - loss: 13.3260 - accuracy: 0.3334 - val_loss: 1.0959 - val_accuracy: 0.3240
Epoch 2/25
36/36 [==============================] - 253s 7s/step - loss: 1.0931 - accuracy: 0.3811 - val_loss: 1.0988 - val_accuracy: 0.3240
Epoch 3/25
36/36 [==============================] - 240s 7s/step - loss: 1.0968 - accuracy: 0.3825 - val_loss: 1.0991 - val_accuracy: 0.3240
Epoch 4/25
36/36 [==============================] - 241s 7s/step - loss: 1.0960 - accuracy: 0.3451 - val_loss: 1.0997 - val_accuracy: 0.3240
Epoch 5/25
36/36 [==============================] - 239s 7s/step - loss: 1.0960 - accuracy: 0.3779 - val_loss: 1.1004 - val_accuracy: 0.3240
Epoch 6/25
36/36 [==============================] - 241s 7s/step - loss: 1.0925 - accuracy: 0.3790 - val_loss: 1.1009 - val_accuracy: 0.3240
Epoch 7/25
36/36 [==============================] - 241s 7s/step - loss: 1.0958 - accuracy: 0.3434 - val_loss: 1.1017 - val_accuracy: 0.3240
Epoch 8/25
3

In [17]:
y_predict = model.predict(X_test)
y_label = np.argmax(y_predict, axis=1)
y_label = y_label.tolist()
print(y_t)
print(y_label)
# plt.plot(y_label, y_t, "o")

k = 0
for i in range(len(y_label)):
  if(y_label[i] == y_t[i]):
    k+=1

print(k / len(y_label))

[0, 0, 2, 2, 1, 2, 2, 0, 0, 0, 2, 0, 1, 1, 2, 2, 1, 0, 2, 2, 0, 0, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 0, 1, 1, 1, 2, 1, 0, 1, 0, 0, 2, 2, 1, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 2, 0, 1, 2, 1, 1, 1, 1, 2, 1, 2, 0, 1, 0, 2, 0, 2, 1, 2, 0, 2, 1, 2, 2, 2, 0, 1, 0, 0, 0, 0, 2, 2, 1, 2, 1, 1, 0, 0, 1, 1, 1, 0, 0, 2, 2, 0, 1, 1, 0, 0, 1, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 2, 1, 1, 0, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 2, 0, 2, 2, 2, 2, 1, 1, 0, 0, 2, 0, 0, 2, 0, 1, 2, 1, 2, 0, 1, 1, 2, 1, 2, 2, 1, 0, 0, 2, 0, 0, 1, 1, 1, 0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0, 2, 1, 1, 2, 1, 1, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,